In [11]:
from scipy.optimize import linprog
import numpy as np
from cvxopt import matrix
from cvxopt import glpk

# The perfect cake recipe that I partially remember

# Equations to solve
# f + e + b + s = 700
# b -0.5s = 0
# f + e <= 450
# e + b <= 300
# -f + e - b + s <= 0

# X matrix
var_list = ['Flour', 'Eggs', 'Butter', 'Sugar']

# Inequality equations, LHS
A_ineq = [[1., 1., 0., 0.], [0., 1., 1., 0.], [-1., 1., -1., 1.]]

# Inequality equations, RHS
B_ineq = [450., 300.,0.]

# Equality equations, LHS
A_eq = [[1., 1., 1., 1.], [0., 0., 1., -0.5]]

# Equality equations, RHS
B_eq = [700., 0]

# Cost function
c = [0., 0., 1., 1.]  # construct a cost function

print('WITHOUT BOUNDS')
res_no_bounds = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq, b_eq=B_eq, method='interior-point')
print(res_no_bounds)



WITHOUT BOUNDS
     con: array([ 5.27055590e-08, -3.78719278e-11])
     fun: 249.99999998121916
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([3.39247208e-08, 6.95661413e+01, 7.24656159e+01])
  status: 0
 success: True
       x: array([302.8994746 , 147.10052537,  83.33333333, 166.66666665])


In [12]:
# these are the bounds that help the solver arrive at a solution 
f_b = [0., 300.]  # limits for flour
e_b = [0., 200.]  # limits for eggs
b_b = [0., 100.]  # limits for butter
s_b = [0., 200.]  # limits for sugar

res_bounds = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq, b_eq=B_eq, bounds=(f_b, e_b, b_b, s_b),
                     method='interior-point')
print('\nWITH BOUNDS')
print(res_bounds)


WITH BOUNDS
     con: array([ 4.53132998e-09, -3.25428573e-12])
     fun: 249.9999999983819
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([2.91322522e-09, 5.30261661e+01, 3.93856656e+01])
  status: 0
 success: True
       x: array([286.35949946, 163.64050053,  83.33333333, 166.66666667])


In [13]:
# Write a parser for SciPy results
def result_parser(res, var_list):
    solve_status = res.success
    if solve_status is True:
        solution_list = res.x
        soln = {var_list[i]: np.round(solution_list[i]) for i in range(len(var_list))}
    else:
        soln = []
    return soln

print('\n')
print('Result (no bounds):\n', result_parser(res_no_bounds, var_list))
print('Result (with bounds):\n', result_parser(res_bounds, var_list))




Result (no bounds):
 {'Flour': 303.0, 'Eggs': 147.0, 'Butter': 83.0, 'Sugar': 167.0}
Result (with bounds):
 {'Flour': 286.0, 'Eggs': 164.0, 'Butter': 83.0, 'Sugar': 167.0}


In [15]:
# formulate problem in terms of GLPK/cvxopt
c_mat, A_ineq_mat, B_ineq_mat, A_eq_mat, B_eq_mat = matrix(c), matrix(A_ineq).T, matrix(B_ineq), matrix(A_eq).T, matrix(B_eq)

# solve the problem
status, solution = glpk.ilp(c_mat, A_ineq_mat, B_ineq_mat, A_eq_mat, B_eq_mat)

if status=='optimal':
    print('solution found')
    print(solution)
else: 
    print(status)

solution found
[ 2.67e+02]
[ 1.83e+02]
[ 8.33e+01]
[ 1.67e+02]



In [26]:
# Adding the extra equality equation
# f + s = 500

# Equality equations, LHS
A_eq_add = A_eq + [[1.,0.,0.,1.]]

# Equality equations, RHS
B_eq_add = B_eq + [500]

# run the SciPy solver without bounds
res_no_bounds_add = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq_add, b_eq=B_eq_add, method='interior-point')

# run the SciPy solver with bounds
res_bounds_add = linprog(c, A_ub=A_ineq, b_ub=B_ineq, A_eq=A_eq_add, b_eq=B_eq_add, bounds=(f_b, e_b, b_b, s_b),
                     method='interior-point')

print('With Additional Eq.\n')
print('Result (no bounds):\n', result_parser(res_no_bounds_add, var_list))
print('Result (with bounds):\n', result_parser(res_bounds_add, var_list))
        

With Additional Eq.

Result (no bounds):
 {'Flour': 333.0, 'Eggs': 117.0, 'Butter': 83.0, 'Sugar': 167.0}
Result (with bounds):
 {'Flour': 300.0, 'Eggs': 100.0, 'Butter': 100.0, 'Sugar': 200.0}


In [27]:
# formulate problem in terms of GLPK/cvxopt
A_eq_add_mat, B_eq_add_mat = matrix(A_eq_add).T, matrix(B_eq_add)

# solve the problem
status_add, solution_add = glpk.ilp(c_mat, A_ineq_mat, B_ineq_mat, A_eq_add_mat, B_eq_add_mat)

if status_add=='optimal':
    print('solution found')
    print(solution_add)
else: 
    print(status_add)

solution found
[ 3.33e+02]
[ 1.17e+02]
[ 8.33e+01]
[ 1.67e+02]

